# Replicating Qubit Results

This notebook attempts to replicate results from this [paper](https://www.science.org/doi/10.1126/sciadv.abl6698).

First problem: when specifying Ω, α, the values calculated by Transmon() are different and use the eigenenergies. Find a way to reverse-engineer the Hamiltonian so they are kept the same as specified.

In [1]:
# imports
import pandas as pd
import numpy as np

from transmon_code.superoperators import *
from transmon_code.transmon import Transmon
from transmon_code.optimise_X90 import *
from transmon_code.simulate import *
from transmon_code.helpers import *

In [2]:
# read data from csv fild
data = pd.read_csv("qubit_data.csv")

transmons = [Transmon(n_levels=8, Ω=data["Ω (MHz)"][i], α=data["α (MHz)"][i], RWA=False, t_decay=data["T1 (μs)"][i], t_dephase=data["T2 (μs)"][i], dt=5e-6) for i in range(len(data))]

data

,Ω (MHz),α (MHz),T1 (μs),T2 (μs),EPG1,EPG2,EPG theoretical
0,3981,-199,106,101,0.000229,0.000164,0.000110
1,4045,-199,159,116,0.000146,0.000215,0.000094
2,4130,-198,179,128,0.000116,0.000131,0.000085
3,4192,-197,151,113,0.000223,0.000216,0.000097


In [3]:
# setup transmon
qubit_no = 0 

t = np.arange(0, 0.05, transmons[qubit_no].dt)

transmons[qubit_no].X90_args = {'A': 15.244446172841954, 'Γ': 0.24, "ω":transmons[qubit_no].Ω, 'A_DRAG': 0.012773896917209865}
transmons[qubit_no].X90_args = {'A': 130.45643689487196, 'Γ': 0.024, 'ω': transmons[qubit_no].Ω, 'A_DRAG': 0.5/transmons[qubit_no].α}
transmons[qubit_no].t_decay = np.inf
transmons[qubit_no].t_dephase = np.inf

# apparently DRAG parameter is 0.5/α to maximise avg gate fidelity and 1/α to minimise leakage errors?

semiranges = {"A":1, "Γ":0.02, "A_DRAG":0.1}
semiranges = {"A":60, "Γ":0, "A_DRAG":0.05}

In [4]:
# train pulse for transmon


transmons[0].X90_args = create_X90_pulse(t, transmons[qubit_no], transmons[qubit_no].X90_args, semiranges, plot=True, N=50)

In [ ]:
# simulates pulse and breaks down error
%matplotlib qt

targ = (basis(transmons[qubit_no].n_levels, 0) - 1j*basis(transmons[qubit_no].n_levels, 1)).unit()
tmp_args = {'A': 400, 'Γ': 0.024, 'ω': transmons[qubit_no].Ω, 'A_DRAG': 0.002}
tmp_args = transmons[qubit_no].X90_args

res, f = simulate(transmons[qubit_no], tmp_args, target=targ, plot=True)

break_down_errors(transmons[qubit_no], transmons[qubit_no].X90_args, res[-1], f)

print(data["EPG theoretical"][qubit_no])